In [1]:
#Clean up existing files and database if code is being executed multiple times
!del hawaii.sqlite 
!del hawaii_measurements_clean.csv 
!del hawaii_stations_clean.csv

C:\Users\Michael Diaz\SqlAlchemy\hawaii.sqlite


The process cannot access the file because it is being used by another process.
Could Not Find C:\Users\Michael Diaz\SqlAlchemy\hawaii_measurements_clean.csv
Could Not Find C:\Users\Michael Diaz\SqlAlchemy\hawaii_stations_clean.csv


In [8]:
# Import Matplot lib and pandas
### BEGIN SOLUTION
import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
#### STEP 1 Reading the Csv files, using Pandas DATA ENGINEERING ###
#Read the Measurments file
#csvfile_measure = 'Resources/cleaned_hi_measurements.csv'
csvfile_measure = 'clean_hawaii_measurements.csv'
df_hawaii_m = pd.read_csv(csvfile_measure)
#Remove NaN form the data
df_hawaii_m = df_hawaii_m.dropna(axis = 0, how='any')

#df_hawaii_m.to_csv('hawaii_measurements_clean.csv', index = False)
df_hawaii_m.to_csv(csvfile_measure, index = False)
df_hawaii_m.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [10]:
#### STEP 1 continued , Reading the Csv files using Pandas, DATA ENGINEERING ###
#Read the Stations file
csvfile_stations = 'clean_hawaii_stations.csv'
df_hawaii_s = pd.read_csv(csvfile_stations)
#Remove NaN form the data
df_hawaii_s = df_hawaii_s.dropna(axis = 0, how='any')

df_hawaii_s.to_csv('clean_hawaii_stations.csv', index = False)
df_hawaii_s.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [11]:
#### Step 2 DATABASE ENGINEERING ####
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()


In [12]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [13]:
conn = engine.connect()

In [14]:
#Create a class for the stations table
class Station(Base):
    __tablename__ = 'station'
    station_id = Column(Integer, primary_key = True)
    station= Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
   

In [15]:
#Create a class for the measurements table
class Measurements(Base):
    __tablename__ = 'measurement'
    measurement_id = Column(Integer, primary_key = True)
    station = Column(String)
    date = Column(String)
    #date = Column(String, primary_key = True)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    

In [16]:
Base.metadata.create_all(engine)


In [17]:
# Load the cleaned station csv file into a pandas dataframe
clean_station_df = pd.read_csv('clean_hawaii_stations.csv')
s_data = clean_station_df.to_dict(orient='records')
print(s_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [18]:
# Load the cleaned station csv file into a pandas dataframe
#clean_measurements_df = pd.read_csv('Resources/cleaned_hi_measurements.csv')
clean_measurements_df = pd.read_csv('clean_hawaii_measurements.csv')
#print(clean_measurements_df.head())
m_data = clean_measurements_df.to_dict(orient='records')
print(m_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-06', 'prcp': 0.0, 'tobs': 73}]


In [19]:
metadata = MetaData(bind=engine)
metadata.reflect()


In [20]:
# Save the reference to the `station` table as a variable called `st`
st = sqlalchemy.Table('station', metadata, autoload=True)

In [21]:
# Save the reference to the `measurement` table as a variable called `me`
me = sqlalchemy.Table('measurement', metadata, autoload=True)

In [22]:
# Use `st.delete()` to remove any pre-existing data, from station table
conn.execute(st.delete())

In [23]:
# Use `me.delete()` to remove any pre-existing data, from measurement table
conn.execute(me.delete())

In [24]:
# Use `st.insert()` to insert the data into the station table
# The SQL table is populated during this step
conn.execute(st.insert(), s_data)

In [25]:
# Use `me.insert()` to insert the data into the measurement table
# The SQL table is populated during this step
conn.execute(me.insert(), m_data)

In [26]:
# Test that the insert works by fetching the first 5 rows from station table. 
conn.execute("select * from station limit 5").fetchall()


[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [27]:
#Also verify the row count from the stations table, show match the rows in the clean station CSV file
res = conn.execute("select count(*) from station;")
print(res.scalar())

9


In [28]:
# Test that the insert works by fetching the first 5 rows from measurement table. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [29]:
#Also verify the row count from the measurements table, show match the rows in the clean measurement CSV file
res = conn.execute("select count(*) from measurement;")
print(res.scalar())

19550
